## **0. Import Libraries**

In [6]:
import os
import math
import random
import numpy as np
from collections import defaultdict, Counter
from sklearn.metrics import classification_report, accuracy_score

## **1. DATA LOADING UTILITIES**

In [ ]:

def load_data(filepath):
    """
    Reads CoNLL-style NER data (Word Tag per line).
    """
    if not os.path.exists(filepath):
        print(f"Warning: File not found at {filepath}")
        return [], []

    sentences = []
    labels = []
    
    with open(filepath, 'r', encoding='utf-8') as f:
        words = []
        tags = []
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    sentences.append(words)
                    labels.append(tags)
                    words = []
                    tags = []
            else:
                try:
                    parts = line.split()
                    # Handle cases where line might have more than 2 columns
                    word = parts[0]
                    tag = parts[-1] 
                    words.append(word)
                    tags.append(tag)
                except ValueError:
                    continue
        
        # Capture last sentence if file doesn't end with newline
        if words:
            sentences.append(words)
            labels.append(tags)
            
    print(f"Loaded {len(sentences)} sentences from {filepath}")
    return sentences, labels

def split_train_valid(sentences, labels, valid_ratio=0.2, seed=42):
    """
    Splits data into training and validation sets.
    """
    random.seed(seed)
    data = list(zip(sentences, labels))
    random.shuffle(data)
    
    split_idx = int(len(data) * (1 - valid_ratio))
    train_data = data[:split_idx]
    valid_data = data[split_idx:]
    
    # Unzip back to lists
    train_sents, train_lbls = zip(*train_data) if train_data else ([], [])
    valid_sents, valid_lbls = zip(*valid_data) if valid_data else ([], [])
    
    return list(train_sents), list(train_lbls), list(valid_sents), list(valid_lbls)

## **2. OPTIMIZED HMM MODEL**

In [ ]:

class OptimizedHMM_NER:
    def __init__(self, alpha=0.1, gamma=2.0, sampling_strategy='balanced'):
        self.trans_probs = defaultdict(lambda: defaultdict(float))
        self.emit_probs = defaultdict(lambda: defaultdict(float))
        self.tags = set()
        self.vocab = set()
        self.START = "<START>"
        self.END = "<END>"
        self.UNK = "<UNK>"
        self.alpha = alpha
        self.gamma = gamma
        self.sampling_strategy = sampling_strategy

    def compute_class_weights(self, labels):
        tag_counts = Counter(tag for seq in labels for tag in seq)
        median_freq = np.median(list(tag_counts.values()))
        weights = {}
        for tag, count in tag_counts.items():
            if tag == 'O':
                weights[tag] = 0.5 
            else:
                weights[tag] = min(5.0, median_freq / (count + 1e-5))
        return weights

    def apply_focal_adjustment(self, prob, tag):
        if tag == 'O': return prob
        focal_weight = self.alpha * (1 - prob) ** self.gamma
        return min(1.0, prob * (1 + focal_weight))

    def balanced_sampling(self, sentences, labels):
        print("Applying balanced sampling...")
        tag_dist = Counter(tag for seq in labels for tag in seq)
        max_count = max(tag_dist.values()) if tag_dist else 0
        
        new_sentences, new_labels = list(sentences), list(labels)
        class_indices = defaultdict(list)

        for idx, tags in enumerate(labels):
            for tag in set(tags):
                if tag != 'O':
                    class_indices[tag].append(idx)

        for tag, indices in class_indices.items():
            if not indices: continue
            target_count = int(max_count * 0.5) # Reduced factor to prevent memory explosion
            current_count = tag_dist[tag]
            
            if current_count < target_count:
                needed = target_count - current_count
                samples = random.choices(indices, k=needed)
                for idx in samples:
                    new_sentences.append(sentences[idx])
                    new_labels.append(labels[idx])
                    
        return new_sentences, new_labels

    def train(self, sentences, labels):
        if self.sampling_strategy == 'balanced':
            sentences, labels = self.balanced_sampling(sentences, labels)

        class_weights = self.compute_class_weights(labels)
        print(f"Training on {len(sentences)} sentences. Class weights computed.")

        trans_counts = defaultdict(lambda: defaultdict(float))
        emit_counts = defaultdict(lambda: defaultdict(float))
        tag_counts = defaultdict(float)

        for sent, tags in zip(sentences, labels):
            prev_tag = self.START
            for word, tag in zip(sent, tags):
                weight = class_weights.get(tag, 1.0)
                emit_counts[tag][word] += weight
                trans_counts[prev_tag][tag] += weight
                tag_counts[tag] += weight
                
                self.tags.add(tag)
                self.vocab.add(word)
                prev_tag = tag
            trans_counts[prev_tag][self.END] += class_weights.get(prev_tag, 1.0)

        # Smoothing and Probability Calculation
        smoothing = 1e-3
        
        # Transitions
        for prev_tag in trans_counts:
            total = sum(trans_counts[prev_tag].values()) + len(self.tags) * smoothing
            for tag in self.tags.union({self.END}):
                prob = (trans_counts[prev_tag].get(tag, 0) + smoothing) / total
                prob = self.apply_focal_adjustment(prob, tag)
                self.trans_probs[prev_tag][tag] = math.log(max(prob, 1e-10))

        # Emissions
        for tag in emit_counts:
            total = sum(emit_counts[tag].values()) + len(self.vocab) * smoothing
            for word in list(emit_counts[tag].keys()): # Iterate only known words
                prob = (emit_counts[tag][word] + smoothing) / total
                prob = self.apply_focal_adjustment(prob, tag)
                self.emit_probs[tag][word] = math.log(max(prob, 1e-10))
            
            # Unknown word handling per tag
            unk_prob = smoothing / total
            unk_prob = self.apply_focal_adjustment(unk_prob, tag)
            self.emit_probs[tag][self.UNK] = math.log(max(unk_prob, 1e-10))

    def viterbi(self, sentence):
        """
        Standard Viterbi implementation with backpointers (O(T*N^2))
        More memory efficient than storing full paths.
        """
        T = len(sentence)
        if T == 0: return []
        
        # V[t][tag] = max log probability
        V = [{} for _ in range(T)]
        # backpointers[t][tag] = previous_tag that led to max prob
        backpointers = [{} for _ in range(T)]

        # Initialization
        for tag in self.tags:
            emit_p = self.emit_probs[tag].get(sentence[0], self.emit_probs[tag][self.UNK])
            trans_p = self.trans_probs[self.START].get(tag, -float('inf'))
            V[0][tag] = trans_p + emit_p
            backpointers[0][tag] = self.START

        # Recursion
        for t in range(1, T):
            word = sentence[t]
            for curr_tag in self.tags:
                emit_p = self.emit_probs[curr_tag].get(word, self.emit_probs[curr_tag][self.UNK])
                
                best_prob = -float('inf')
                best_prev = None
                
                # Iterate over previous tags to find best transition
                for prev_tag in self.tags:
                    # Previous path prob + Transition prob
                    prob = V[t-1][prev_tag] + self.trans_probs[prev_tag].get(curr_tag, -float('inf'))
                    
                    if prob > best_prob:
                        best_prob = prob
                        best_prev = prev_tag
                
                V[t][curr_tag] = best_prob + emit_p
                backpointers[t][curr_tag] = best_prev

        # Termination
        best_final_prob = -float('inf')
        best_last_tag = None
        
        for tag in self.tags:
            prob = V[T-1][tag] + self.trans_probs[tag].get(self.END, -float('inf'))
            if prob > best_final_prob:
                best_final_prob = prob
                best_last_tag = tag

        # Backtracking
        if best_last_tag is None: return ["O"] * T # Fallback
        
        best_path = [best_last_tag]
        curr_tag = best_last_tag
        
        for t in range(T-1, 0, -1):
            prev_tag = backpointers[t][curr_tag]
            best_path.append(prev_tag)
            curr_tag = prev_tag
            
        return best_path[::-1]

    def predict(self, sentences):
        return [self.viterbi(sent) for sent in sentences]

## **3. EVALUATION UTILITIES**

In [ ]:

def analyze_ner_errors(true_tags, pred_tags, top_k=10):
    error_types = Counter()
    for true_seq, pred_seq in zip(true_tags, pred_tags):
        for true, pred in zip(true_seq, pred_seq):
            if true != pred:
                error_types[f"{true} -> {pred}"] += 1
    return error_types.most_common(top_k)

def print_misclassified_sentences(sentences, true_labels, pred_labels, num_samples=3):
    print(f"\n--- Top {num_samples} Misclassified Sentences ---")
    count = 0
    for idx, (tokens, true, pred) in enumerate(zip(sentences, true_labels, pred_labels)):
        if true != pred:
            print(f"\nSentence {idx + 1}:")
            print(f"{'WORD':<15} {'TRUE':<10} {'PRED':<10}")
            print("-" * 35)
            for t, tr, pr in zip(tokens, true, pred):
                mark = "Wrong" if tr != pr else ""
                print(f"{t:<15} {tr:<10} {pr:<10} {mark}")
            count += 1
            if count >= num_samples: break

## **4. MAIN EXECUTION**

In [ ]:


# CONFIGURATION: Update this path to where your files are located
# On Kaggle, if you uploaded files, it might be "../input/dataset_name/"
# If you just dragged files into the files pane, try "./"
DATA_DIR = "/kaggle/input/dataset" 
TRAIN_FILE = os.path.join(DATA_DIR, "train.txt")
TEST_FILE = os.path.join(DATA_DIR, "test.txt")

# 1. Load Data
print(">>> Loading Data...")
train_sents, train_tags = load_data(TRAIN_FILE)
test_sents, test_tags = load_data(TEST_FILE)

if not train_sents:
    print("Error: No training data found. Please check file paths.")
else:
    # 2. Split Validation
    print(">>> Splitting Train/Validation...")
    train_text_split, train_label_split, valid_text, valid_label = split_train_valid(train_sents, train_tags)

    # 3. Train Model
    print(">>> Training Optimized HMM...")
    model = OptimizedHMM_NER(alpha=0.1, gamma=2.0, sampling_strategy='balanced')
    model.train(train_text_split, train_label_split)

    # 4. Evaluate on Validation
    print("\n>>> Evaluating on Validation Set...")
    pred_valid = model.predict(valid_text)
    
    true_valid_flat = [tag for seq in valid_label for tag in seq]
    pred_valid_flat = [tag for seq in pred_valid for tag in seq]
    
    # Filter labels to avoid cluttering report with 'O' if desired, or keep all
    target_labels = sorted(list({t for t in true_valid_flat if t != 'O'}))
    
    print(classification_report(true_valid_flat, pred_valid_flat, zero_division=0))
    print(f"Validation Accuracy: {accuracy_score(true_valid_flat, pred_valid_flat):.4f}")

    # 5. Evaluate on Test (if available)
    if test_sents:
        print("\n>>> Evaluating on Test Set...")
        pred_test = model.predict(test_sents)
        
        true_test_flat = [tag for seq in test_tags for tag in seq]
        pred_test_flat = [tag for seq in pred_test for tag in seq]
        
        print(classification_report(true_test_flat, pred_test_flat, zero_division=0))
        
        # Error Analysis
        print("\n>>> Top NER Errors:")
        top_errors = analyze_ner_errors(test_tags, pred_test)
        for err, count in top_errors:
            print(f"{err}: {count}")
            
        # Show Examples
        print_misclassified_sentences(test_sents, test_tags, pred_test)

>>> Loading Data...
Loaded 13486 sentences from /kaggle/input/dataset/train.txt
Loaded 3372 sentences from /kaggle/input/dataset/test.txt
>>> Splitting Train/Validation...
>>> Training Optimized HMM...
Applying balanced sampling...
Training on 878271 sentences. Class weights computed.

>>> Evaluating on Validation Set...
              precision    recall  f1-score   support

       B-LOC       0.71      0.79      0.75      1013
      B-MISC       0.29      0.95      0.44        38
       B-ORG       0.57      0.78      0.66       209
       B-PER       0.86      0.89      0.88      1172
       I-LOC       0.68      0.73      0.70       461
      I-MISC       0.29      0.95      0.44        38
       I-ORG       0.59      0.65      0.62       345
       I-PER       0.87      0.92      0.89       604
           O       0.99      0.98      0.99     54974

    accuracy                           0.97     58854
   macro avg       0.65      0.85      0.71     58854
weighted avg       0.98    